# 06 - Continuous Retraining Pipeline

**Author:** Hector Carbajal  
**Version:** 1.0  
**Last Updated:** 2026-02

---

## Purpose
This notebook covers **MLOps maintenance practices**:
- **Drift detection**: Monitor feature distributions and model performance
- **Retraining triggers**: Automatic model updates when drift is detected
- **Model versioning**: Save and track model versions (v001, v002, etc.)
- **CI/CD integration**: Update production risk gates with new models

## Inputs
- Historical training data
- New production data (simulated)

## Outputs
- `models/catboost_classifier_v002.cbm` - Retrained model version
- Drift analysis reports

## Retraining Trigger Criteria
- Feature drift > 15% from historical baseline
- Model accuracy drops > 5% from baseline
- New data volume threshold reached

---

In [1]:
# [1] Setup & Imports
# --------------------------------------------------------------------------
import os
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier 
from sklearn.metrics import roc_auc_score
import joblib 
import sys

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Add project root to path and import config
current_dir = Path.cwd()
project_root = current_dir.parents[0] if current_dir.name == "notebooks" else current_dir
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.config import PROCESSED_DATA_DIR, MODELS_DIR, PROJECT_ROOT

# For backward compatibility
DATA_DIR = PROCESSED_DATA_DIR
MODEL_DIR = MODELS_DIR
MODEL_PATH = MODEL_DIR / "catboost_classifier_v001.cbm"

print(f"✅ Configuration loaded")
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Model directory: {MODEL_DIR}")

✅ Configuration loaded
📁 Project root: /Users/hc/Documents/projects/roku-patch-stability-analytics
📁 Data directory: /Users/hc/Documents/projects/roku-patch-stability-analytics/data/Processed
📁 Model directory: /Users/hc/Documents/projects/roku-patch-stability-analytics/models


In [2]:
# [2] Load Historical Data and Simulate New Data Arrival
# --------------------------------------------------------------------------

# --- A. Load Historical (Training) Data ---
# This is the original synthetic feature file (simulates your entire historical dataset)
historical_df = pd.read_csv(DATA_DIR / "synthetic_firmware_features.csv")

# Ensure target column is present using 75th percentile threshold (like notebook 03)
if 'high_risk_flag' not in historical_df.columns:
    # Use 75th percentile threshold (matching notebook 03)
    threshold = historical_df['error_rate_per_10k'].quantile(0.75)
    historical_df['high_risk_flag'] = (historical_df['error_rate_per_10k'] >= threshold).astype(int)
    print(f"Created high_risk_flag using threshold: {threshold:.0f}")

print(f"Target distribution in historical data:")
print(historical_df['high_risk_flag'].value_counts())

# --- B. Simulate New Data Arrival (New Patches + Known Outcomes) ---
# In a real environment, this data would come from the live deployment and monitoring systems
# We'll simulate fetching 50 new patch records (with known outcomes)
new_data_df = historical_df.sample(n=50, random_state=99).copy()
new_data_df['firmware_version'] = new_data_df['firmware_version'].astype(str) + '.NEW'

# --- C. Combine for Retraining ---
retrain_df = pd.concat([historical_df, new_data_df], ignore_index=True)

print(f"\nHistorical (Training) Size: {historical_df.shape[0]} rows")
print(f"New Data Size: {new_data_df.shape[0]} rows")
print(f"Combined Retraining Dataset Size: {retrain_df.shape[0]} rows")
print(f"\nTarget distribution for retraining:")
print(retrain_df['high_risk_flag'].value_counts())

Created high_risk_flag using threshold: 2835
Target distribution in historical data:
high_risk_flag
0    500
1    500
Name: count, dtype: int64

Historical (Training) Size: 1000 rows
New Data Size: 50 rows
Combined Retraining Dataset Size: 1050 rows

Target distribution for retraining:
high_risk_flag
0    528
1    522
Name: count, dtype: int64


In [ ]:
# [3] Drift Analysis and Retraining Trigger Check
# --------------------------------------------------------------------------

# 1. Check for Population Drift (Feature Change)
# Compares the mean of the most important feature (code_churn_score)
hist_mean_churn = historical_df['code_churn_score'].mean()
hist_std_churn = historical_df['code_churn_score'].std()
new_mean_churn = new_data_df['code_churn_score'].mean()

# Calculate drift as percentage change from historical baseline
# This is more intuitive than standard deviation-based thresholds for business audiences
drift_pct = abs(new_mean_churn - hist_mean_churn) / hist_mean_churn * 100 if hist_mean_churn != 0 else 0
DRIFT_THRESHOLD_PCT = 15.0  # 15% drift triggers retraining
RETRAIN_TRIGGERED = drift_pct > DRIFT_THRESHOLD_PCT

print(f"Historical Mean Churn: {hist_mean_churn:.4f} (std: {hist_std_churn:.4f})")
print(f"New Data Mean Churn: {new_mean_churn:.4f}")
print(f"Feature Drift (Churn): {drift_pct:.2f}%")

if RETRAIN_TRIGGERED:
    print(f"\n🚨 RETRAINING TRIGGERED: (Drift of {drift_pct:.2f}% exceeds threshold of {DRIFT_THRESHOLD_PCT:.1f}%.)")
else:
    print(f"\n✅ Drift ({drift_pct:.2f}%) within acceptable limits ({DRIFT_THRESHOLD_PCT:.1f}%). No retraining needed.")

In [4]:
# [4] Model Retraining and Versioning
# --------------------------------------------------------------------------

if RETRAIN_TRIGGERED:
    
    FEATURES = [
        "code_churn_score",
        "previous_version_error_rate",
        "avg_device_age_days",
        "is_hotfix",
        "patch_security",
    ]
    TARGET = "high_risk_flag"

    # Define X and y for the combined retraining dataset
    X_full = retrain_df[FEATURES]
    y_full = retrain_df[TARGET]

    # Split into train and test sets for honest evaluation
    X_train, X_test, y_train, y_test = train_test_split(
        X_full, y_full, test_size=0.2, random_state=42, stratify=y_full
    )

    # --- Retrain the CatBoost Classifier ---
    cbc_v002 = CatBoostClassifier(
        iterations=500,
        learning_rate=0.01,
        loss_function='Logloss',
        verbose=0,
        random_seed=42
    )

    cbc_v002.fit(X_train, y_train)
    
    # --- Versioning and Saving ---
    # In a real system, we would increment the version number dynamically
    new_model_path = MODEL_DIR / "catboost_classifier_v002.cbm"
    cbc_v002.save_model(new_model_path)
    
    # Evaluate on holdout set
    test_auc = roc_auc_score(y_test, cbc_v002.predict_proba(X_test)[:, 1])
    
    print(f"\n✅ Retraining complete. New model saved as: {new_model_path.name}")
    print(f"New model ROC AUC (Holdout Set): {test_auc:.4f}")

    # Update the CI Gate to use the new version in production
    print("\n--- NEXT ACTION: Update CI Gate to load 'catboost_classifier_v002.cbm' ---")
else:
    print("Model version v001 remains active.")


✅ Retraining complete. New model saved as: catboost_classifier_v002.cbm
New model ROC AUC (Holdout Set): 0.8919

--- NEXT ACTION: Update CI Gate to load 'catboost_classifier_v002.cbm' ---


## Validation Visualizations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Create retraining pipeline validation visualizations using ACTUAL computed values
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Notebook 06 - Continuous Retraining Pipeline Validation', fontsize=16, fontweight='bold')

# 1. Feature Drift Comparison (ACTUAL data from this run)
features_to_compare = ['code_churn_score', 'previous_version_error_rate', 'avg_device_age_days']
hist_means = [historical_df[f].mean() for f in features_to_compare]
new_means = [new_data_df[f].mean() for f in features_to_compare]
drift_pcts = [abs(n - h) / h * 100 if h != 0 else 0 for h, n in zip(hist_means, new_means)]

x = np.arange(len(features_to_compare))
width = 0.35
bars1 = axes[0, 0].bar(x - width/2, hist_means, width, label='Historical', color='steelblue', alpha=0.8)
bars2 = axes[0, 0].bar(x + width/2, new_means, width, label='New Data', color='coral', alpha=0.8)
axes[0, 0].set_xlabel('Feature')
axes[0, 0].set_ylabel('Mean Value')
axes[0, 0].set_title('Feature Distribution: Historical vs New Data')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(['Code Churn', 'Prev Error Rate', 'Device Age'], fontsize=9)
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3, axis='y')

# 2. Drift Percentage by Feature (ACTUAL)
colors_drift = ['red' if d > DRIFT_THRESHOLD_PCT else 'green' for d in drift_pcts]
bars = axes[0, 1].barh(['Code Churn', 'Prev Error Rate', 'Device Age'], drift_pcts, color=colors_drift, edgecolor='black')
axes[0, 1].axvline(DRIFT_THRESHOLD_PCT, color='red', linestyle='--', linewidth=2, label=f'Threshold: {DRIFT_THRESHOLD_PCT}%')
axes[0, 1].set_xlabel('Drift (%)')
axes[0, 1].set_title('Feature Drift Analysis (ACTUAL)')
axes[0, 1].legend()
for bar, pct in zip(bars, drift_pcts):
    axes[0, 1].text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2, f'{pct:.1f}%', va='center', fontsize=10)

# 3. Retraining Decision Summary (ACTUAL from this run)
decision_data = {
    'Drift Detected': 1 if RETRAIN_TRIGGERED else 0,
    'Model Retrained': 1 if RETRAIN_TRIGGERED else 0,
    'Version Updated': 1 if RETRAIN_TRIGGERED else 0
}
colors_decision = ['green' if v == 1 else 'lightgray' for v in decision_data.values()]
axes[0, 2].bar(decision_data.keys(), decision_data.values(), color=colors_decision, edgecolor='black')
axes[0, 2].set_ylabel('Status (1=Yes, 0=No)')
axes[0, 2].set_title('Retraining Pipeline Decision (This Run)')
axes[0, 2].set_ylim([0, 1.5])

# 4. Target Class Distribution Comparison (ACTUAL)
hist_class_dist = historical_df['high_risk_flag'].value_counts(normalize=True).sort_index()
new_class_dist = new_data_df['high_risk_flag'].value_counts(normalize=True).sort_index()
x = np.arange(2)
width = 0.35
axes[1, 0].bar(x - width/2, hist_class_dist.values, width, label='Historical', color='steelblue', alpha=0.8)
axes[1, 0].bar(x + width/2, new_class_dist.values, width, label='New Data', color='coral', alpha=0.8)
axes[1, 0].set_xlabel('Risk Class')
axes[1, 0].set_ylabel('Proportion')
axes[1, 0].set_title('Target Class Distribution Stability')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(['Low Risk', 'High Risk'])
axes[1, 0].legend()

# 5. Model Performance (ACTUAL if available)
if RETRAIN_TRIGGERED and 'test_auc' in dir():
    # Use actual training performance
    metrics = {'Train AUC': roc_auc_score(y_train, cbc_v002.predict_proba(X_train)[:, 1]),
               'Test AUC': test_auc}
    bars = axes[1, 1].bar(metrics.keys(), metrics.values(), color=['steelblue', 'coral'], edgecolor='black')
    axes[1, 1].set_ylim([0, 1.0])
    for bar in bars:
        axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                       f'{bar.get_height():.3f}', ha='center', fontsize=11, fontweight='bold')
else:
    axes[1, 1].text(0.5, 0.5, 'No retraining performed\n(drift below threshold)', 
                   ha='center', va='center', transform=axes[1, 1].transAxes, fontsize=12)
axes[1, 1].set_ylabel('AUC Score')
axes[1, 1].set_title('Retrained Model Performance')
axes[1, 1].grid(alpha=0.3, axis='y')

# 6. Data Volume Summary (ACTUAL)
volume_data = {
    'Historical\nRecords': len(historical_df),
    'New\nRecords': len(new_data_df),
    'Combined\nTraining': len(retrain_df)
}
colors_vol = ['steelblue', 'coral', 'green']
bars = axes[1, 2].bar(volume_data.keys(), volume_data.values(), color=colors_vol, edgecolor='black')
axes[1, 2].set_ylabel('Record Count')
axes[1, 2].set_title('Data Volume for Retraining')
for bar in bars:
    axes[1, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10, 
                   f'{int(bar.get_height()):,}', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Print validation summary using ACTUAL computed values
print("\n" + "="*60)
print("VALIDATION SUMMARY - Continuous Retraining Pipeline (ACTUAL)")
print("="*60)
print(f"\nDrift Analysis:")
print(f"  Code Churn drift: {drift_pcts[0]:.2f}%")
print(f"  Prev Error Rate drift: {drift_pcts[1]:.2f}%")
print(f"  Device Age drift: {drift_pcts[2]:.2f}%")
print(f"  Threshold: {DRIFT_THRESHOLD_PCT:.1f}%")
print(f"  Status: {'🚨 DRIFT DETECTED' if RETRAIN_TRIGGERED else '✅ Normal'}")
print(f"\nData Volume:")
print(f"  Historical records: {len(historical_df):,}")
print(f"  New records: {len(new_data_df):,}")
print(f"  Combined training set: {len(retrain_df):,}")
if RETRAIN_TRIGGERED and 'test_auc' in dir():
    print(f"\nModel Performance (v002):")
    print(f"  Test AUC: {test_auc:.4f}")
print(f"\nPipeline Status: {'Retraining completed ✅' if RETRAIN_TRIGGERED else 'No retraining needed ✅'}")
print("="*60)